<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Collect-S&amp;P-500-Companies" data-toc-modified-id="Collect-S&amp;P-500-Companies-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Collect S&amp;P 500 Companies</a></span></li><li><span><a href="#Example-code" data-toc-modified-id="Example-code-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Example code</a></span></li><li><span><a href="#Stock-Prices" data-toc-modified-id="Stock-Prices-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Stock Prices</a></span></li><li><span><a href="#Calculate-Correlation" data-toc-modified-id="Calculate-Correlation-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Calculate Correlation</a></span></li></ul></div>

In [1]:
# Import libraries 
import pandas as pd
import os

from datetime import datetime
from urllib.request import urlopen, Request
from bs4 import BeautifulSoup

%matplotlib inline

# Collect S&P 500 Companies

In [2]:
table = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
sandp_df = table[0]

#sandp_df.to_csv('data/S&P500-Info.csv')
#sandp_df.to_csv("data/S&P500-Symbols.csv", columns=['Symbol'])

#https://medium.com/wealthy-bytes/5-lines-of-python-to-automate-getting-the-s-p-500-95a632e5e567

In [3]:
sandp_df.head(5)
# so the symbol is the same as the corresponding stock ticker. 
# It will be used for parsing news results that reference the company that made the headlines.

,Symbol,Security,SEC filings,GICS Sector,GICS Sub-Industry,Headquarters Location,Date first added,CIK,Founded
0,MMM,3M Company,reports,Industrials,Industrial Conglomerates,"St. Paul, Minnesota",1976-08-09,66740,1902
1,ABT,Abbott Laboratories,reports,Health Care,Health Care Equipment,"North Chicago, Illinois",1964-03-31,1800,1888
2,ABBV,AbbVie Inc.,reports,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
3,ABMD,Abiomed,reports,Health Care,Health Care Equipment,"Danvers, Massachusetts",2018-05-31,815094,1981
4,ACN,Accenture,reports,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,1467373,1989


# Example code

Taken from https://towardsdatascience.com/sentiment-analysis-of-stocks-from-financial-news-using-python-82ebdcefb638

In [4]:
finwiz_url = 'https://finviz.com/quote.ashx?t='

news_tables = {}
tickers = sandp_df['Symbol']

for ticker in tickers:
    url = finwiz_url + ticker
    req = Request(url=url, headers={'user-agent': 'my-app/0.0.1'}) 
    try:
        response = urlopen(req)   
        html = BeautifulSoup(response)  # Read the contents of the file into 'html'
        news_table = html.find(id='news-table') # Find 'news-table' in the Soup and load it into 'news_table'
        news_tables[ticker] = news_table # Add the table to our dictionary
    except:
        continue

In [23]:
parsed_news = []

# Iterate through the news
for file_name, news_table in news_tables.items():
    # Iterate through all tr tags in 'news_table'
    for x in news_table.findAll('tr'):
        # read the text from each tr tag into text
        # get text from a only
        text = x.a.get_text() 
        # splite text in the td tag into a list 
        date_scrape = x.td.text.split()
        
        # get news media company
        news = x.span.get_text()
        
        # if the length of 'date_scrape' is 1, load 'time' as the only element
        if len(date_scrape) == 1:
            time = date_scrape[0]
            
        # else load 'date' as the 1st element and 'time' as the second    
        else:
            date = date_scrape[0]
            time = date_scrape[1]
            
        # Extract the ticker from the file name, get the string up to the 1st '_'  
        ticker = file_name.split('_')[0]
        
        # Append ticker, date, time and headline as a list to the 'parsed_news' list
        parsed_news.append([ticker, date, time, text, news])
        
# Set column names
columns = ['ticker', 'date', 'time', 'headline', 'news']

# Convert the parsed_news list into a DataFrame called 'parsed_and_scored_news'
parsed_news_updated = pd.DataFrame(parsed_news, columns=columns)
parsed_news_updated.head(5)

,ticker,date,time,headline,news
0,MMM,Feb-12-21,05:12PM,First Eagle Investment's Top 4th-Quarter Trades,GuruFocus.com
1,MMM,Feb-12-21,10:10AM,Why 3M Is a Retiree's Dream Stock,Motley Fool
2,MMM,Feb-12-21,07:45AM,3 Top Value Stocks to Buy Right Now,Motley Fool
3,MMM,Feb-10-21,06:00AM,How a $1.9B Bond Fund Finds Opportunity in Mar...,Barrons.com
4,MMM,Feb-09-21,02:35PM,3M Announces Upcoming Investor Events,PR Newswire


In [24]:
#need to tokenize each words within the headlines to improve the sentiment score.

import re
import nltk
nltk.download('stopwords')
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

def regex(x):
    special_chars_p = "[.®'&$’\"\-()#@!?/:]"
    s1 = re.sub(special_chars_p, '', x)  
    return(s1)

parsed_news_updated['headline'] = parsed_news_updated['headline'].apply(regex)

stemmer = PorterStemmer()

def stem_sentences(sentence):
    tokens = sentence.lower().split()
    stemmed_tokens = [stemmer.stem(token) for token in tokens]
    return ' '.join(stemmed_tokens)

parsed_news_updated_stem = parsed_news_updated.copy()
parsed_news_updated_stem['headline'] = parsed_news_updated_stem['headline'].apply(stem_sentences)

stop=stopwords.words('english')

parsed_news_updated['headline'].apply(lambda x: [item for item in x if item not in stop])
parsed_news_updated_stem['headline'].apply(lambda x: [item for item in x if item not in stop])

parsed_news_updated['headline'] = parsed_news_updated['headline'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)])) 
parsed_news_updated_stem['headline'] = parsed_news_updated_stem['headline'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)])) 
parsed_news_updated.head()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,ticker,date,time,headline,news
0,MMM,Feb-12-21,05:12PM,First Eagle Investments Top 4thQuarter Trades,GuruFocus.com
1,MMM,Feb-12-21,10:10AM,Why 3M Is Retirees Dream Stock,Motley Fool
2,MMM,Feb-12-21,07:45AM,3 Top Value Stocks Buy Right Now,Motley Fool
3,MMM,Feb-10-21,06:00AM,How 19B Bond Fund Finds Opportunity Market Tur...,Barrons.com
4,MMM,Feb-09-21,02:35PM,3M Announces Upcoming Investor Events,PR Newswire


In [25]:
# NLTK VADER for sentiment analysis (unstem)
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment import SentimentAnalyzer
from nltk.sentiment.vader import SentimentIntensityAnalyzer

# Instantiate the sentiment intensity analyzer
vader = SentimentIntensityAnalyzer()

# Iterate through the headlines and get the polarity scores using vader
scores = parsed_news_updated['headline'].apply(vader.polarity_scores).tolist()

# Convert the 'scores' list of dicts into a DataFrame
scores_df = pd.DataFrame(scores)

# Join the DataFrames of the news and the list of dicts
parsed_and_scored_news = parsed_news_updated.join(scores_df, rsuffix='_right')

# Convert the date column from string to datetime
parsed_and_scored_news['date'] = pd.to_datetime(parsed_and_scored_news.date).dt.date

parsed_and_scored_news.head(10)

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


,ticker,date,time,headline,news,neg,neu,pos,compound
0,MMM,2021-02-12,05:12PM,First Eagle Investments Top 4thQuarter Trades,GuruFocus.com,0.000,0.735,0.265,0.2023
1,MMM,2021-02-12,10:10AM,Why 3M Is Retirees Dream Stock,Motley Fool,0.000,0.714,0.286,0.2500
2,MMM,2021-02-12,07:45AM,3 Top Value Stocks Buy Right Now,Motley Fool,0.000,0.488,0.512,0.4939
3,MMM,2021-02-10,06:00AM,How 19B Bond Fund Finds Opportunity Market Tur...,Barrons.com,0.221,0.531,0.248,0.0772
4,MMM,2021-02-09,02:35PM,3M Announces Upcoming Investor Events,PR Newswire,0.000,1.000,0.000,0.0000
5,MMM,2021-02-09,11:07AM,New 3M Polisher ST reduces number biopharma ma...,PR Newswire,0.000,0.874,0.126,0.0772
6,MMM,2021-02-09,11:07AM,New 3M Polisher ST reduces number biopharma ma...,CNW Group,0.000,0.874,0.126,0.0772
7,MMM,2021-02-08,10:30AM,3 Stocks Buy With Dividends Yielding More Than 3%,Motley Fool,0.000,1.000,0.000,0.0000
8,MMM,2021-02-06,09:12AM,30 Dividend Kings 2021 Part III,Insider Monkey,0.000,1.000,0.000,0.0000
9,MMM,2021-02-06,07:30AM,"3M, Yum Brands, Other Companies That Raised Th...",Barrons.com,0.000,1.000,0.000,0.0000


In [26]:
# NLTK VADER for sentiment analysis (stem)

# Instantiate the sentiment intensity analyzer
vader = SentimentIntensityAnalyzer()

# Iterate through the headlines and get the polarity scores using vader
scores_stem = parsed_news_updated_stem['headline'].apply(vader.polarity_scores).tolist()

# Convert the 'scores' list of dicts into a DataFrame
scores_stem_df = pd.DataFrame(scores_stem)

# Join the DataFrames of the news and the list of dicts
parsed_and_scored_news_stem = parsed_news_updated_stem.join(scores_stem_df, rsuffix='_right')

# Convert the date column from string to datetime
parsed_and_scored_news_stem['date'] = pd.to_datetime(parsed_and_scored_news_stem.date).dt.date

parsed_and_scored_news_stem.head(10)

,ticker,date,time,headline,news,neg,neu,pos,compound
0,MMM,2021-02-12,05:12PM,first eagl invest top 4thquarter trade,GuruFocus.com,0.000,0.735,0.265,0.2023
1,MMM,2021-02-12,10:10AM,whi 3m retire dream stock,Motley Fool,0.000,0.667,0.333,0.2500
2,MMM,2021-02-12,07:45AM,3 top valu stock buy right,Motley Fool,0.000,0.690,0.310,0.2023
3,MMM,2021-02-10,06:00AM,19b bond fund find opportun market turmoil,Barrons.com,0.294,0.706,0.000,-0.3612
4,MMM,2021-02-09,02:35PM,3m announc upcom investor event,PR Newswire,0.000,1.000,0.000,0.0000
5,MMM,2021-02-09,11:07AM,new 3m polish st reduc number biopharma manufa...,PR Newswire,0.000,0.874,0.126,0.0772
6,MMM,2021-02-09,11:07AM,new 3m polish st reduc number biopharma manufa...,CNW Group,0.000,0.874,0.126,0.0772
7,MMM,2021-02-08,10:30AM,3 stock buy dividend yield 3%,Motley Fool,0.000,1.000,0.000,0.0000
8,MMM,2021-02-06,09:12AM,30 dividend king 2021 part iii,Insider Monkey,0.000,1.000,0.000,0.0000
9,MMM,2021-02-06,07:30AM,"3m, yum brands, compani rais dividend thi week",Barrons.com,0.000,1.000,0.000,0.0000


# Stock Prices

In [27]:
# Get S&P 500 prices
# source: https://www.spglobal.com/spdji/en/indices/equity/sp-500/#overview

df_sp = pd.read_csv('data/S&P500_5years.csv', usecols=[0,1]) # Use only first 2 columns
df_sp.columns = ['date', 'price']
df_sp['date'] = pd.to_datetime(df_sp['date'])
df_sp.head()

,date,price
0,2016-11-02,1829.08
1,2016-12-02,1864.78
2,2016-02-16,1895.58
3,2016-02-17,1926.82
4,2016-02-18,1917.83


In [28]:
# Get S&P 500 individual stock prices

# Create a function to get stock price given a ticker 
def get_stock_price(ticker, start, end):
    '''Get prices of a stock in a given period.
    
    Args:
        ticker (str): ticker of a company 
        start (str): date in format of 'YYYY-MM-DD'
        end (str): date in format of 'YYYY-MM-DD'
    
    Returns:
        A DataFrame containing open, high, low, close, volume, dividends, stock splits
    '''
    import yfinance as yf
    
    ticker = yf.Ticker(ticker)
    data = ticker.history(start=start, end=end)
    data.reset_index(level=0, inplace=True)
    return data 


"\n# Get a list of 505 stocks from S&P 500\nsp500 = sandp_df['Symbol'].unique()\nstart = parsed_and_scored_news['date'].min()\nend = parsed_and_scored_news['date'].max()\n\n# Iterate through each stock to get price\ndf_stock = pd.DataFrame()\n\nfor ticker in sp500:\n    data = get_stock_price(ticker, start, end)\n    data['ticker'] = ticker \n    df_stock = pd.concat([df_stock, data], axis=0)\n    \n"

In [29]:
# Define function to generate merged dataframe (merged by date)
# columns: ['ticker', 'date', 'time', 'headline', 'news', 'neg', 'neu', 'pos', 'compound', 'open', 'close', 'change']
# scored dataframe should be the input (not sure if works for scored sentiment other than Vader)


def generate_final_df(scored_df):
    # Get a list of 505 stocks from S&P 500
    sp500 = sandp_df['Symbol'].unique()
    start = scored_df['date'].min()
    end = scored_df['date'].max()
    
    # Iterate through each stock to get price
    df_stock = pd.DataFrame()
    for ticker in sp500:
        data = get_stock_price(ticker, start, end)
        data['ticker'] = ticker
        df_stock = pd.concat([df_stock, data], axis=0)
        
    # Change all columns names to lowercase  
    df_stock.columns = df_stock.columns.str.lower()
    
    # Convert timestamp to date
    df_stock['date'] = df_stock['date'].apply(datetime.date)
    
    # Reset index
    df_stock.reset_index(drop=True, inplace=True)
    
    # Merge stock price info and sentiment scores
    df_merged = scored_df.merge(df_stock.loc[:, ['date', 'ticker', 'open', 'close']], on=['date', 'ticker'])
    # Add column: price change
    df_merged['change'] = df_merged['close'] - df_merged['open']
    
    return df_merged

In [30]:
# Generate final df for unstem and stem

df_final_unstem = generate_final_df(parsed_and_scored_news)
df_final_stem = generate_final_df(parsed_and_scored_news_stem)

- BRK.B: No data found, symbol may be delisted
- BF.B: No data found for this date range, symbol may be delisted
- BRK.B: No data found, symbol may be delisted
- BF.B: No data found for this date range, symbol may be delisted


# Calculate Correlation 

In [50]:
# Calculate pearson correlation coef between sentiment score and price for each news media
scores_close_unstem = df_final_unstem.groupby('news')[['compound', 'close']].corr().unstack().iloc[:, 1].sort_values(ascending=False)
scores_close_stem = df_final_stem.groupby('news')[['compound', 'close']].corr().unstack().iloc[:, 1].sort_values(ascending=False)
scores_change_unstem = df_final_unstem.groupby('news')[['compound', 'change']].corr().unstack().iloc[:, 1].sort_values(ascending=False)
scores_change_stem = df_final_stem.groupby('news')[['compound', 'change']].corr().unstack().iloc[:, 1].sort_values(ascending=False)


# https://stackoverflow.com/questions/28988627/pandas-correlation-groupby

news
 Zacks Small Cap Research           1.000000
 Skift                              1.000000
 The Telegraph                      1.000000
 ETF.com                            1.000000
 CorpGov.com                        0.745148
                                      ...   
 Exec Edge                               NaN
 GOBankingRates                          NaN
 Market Exclusive                        NaN
 Schaeffer's Investment Research         NaN
[email protected]                        NaN
Name: (compound, change), Length: 66, dtype: float64

In [51]:
# Calculate spearman correlation coef between sentiment score and price for each news media
scores_close_unstem = df_final_unstem.groupby('news')[['compound', 'close']].corr(method='spearman').unstack().iloc[:, 1].sort_values(ascending=False)
scores_close_stem = df_final_stem.groupby('news')[['compound', 'close']].corr(method='spearman').unstack().iloc[:, 1].sort_values(ascending=False)
scores_change_unstem = df_final_unstem.groupby('news')[['compound', 'change']].corr(method='spearman').unstack().iloc[:, 1].sort_values(ascending=False)
scores_change_stem = df_final_stem.groupby('news')[['compound', 'change']].corr(method='spearman').unstack().iloc[:, 1].sort_values(ascending=False)


In [48]:
# Pearson's Correlation Coefficient as Dataframe
pearson_corr = pd.DataFrame({'variable' : ['close', 'change'], 'unstem' : [df_final_unstem[['compound', 'close']].corr().iloc[0,1], df_final_unstem[['compound', 'change']].corr().iloc[0,1]], 'stem' : [df_final_stem[['compound', 'close']].corr().iloc[0,1], df_final_stem[['compound', 'change']].corr().iloc[0,1]]})
pearson_corr

,variable,unstem,stem
0,change,0.008800,0.011299
1,close,-0.000116,0.018769


In [47]:
# Spearman's rank correlation
spearman_corr = pd.DataFrame({'variable' : ['close', 'change'], 'unstem' : [df_final_unstem[['compound', 'close']].corr(method='spearman').iloc[0,1], df_final_unstem[['compound', 'change']].corr(method='spearman').iloc[0,1]], 'stem' : [df_final_stem[['compound', 'close']].corr(method='spearman').iloc[0,1], df_final_stem[['compound', 'change']].corr(method='spearman').iloc[0,1]]})
spearman_corr

,variable,unstem,stem
0,change,0.015127,0.014860
1,close,0.026481,0.049613


Vader Sentiment Analysis

Performed Pearson's Correlation Coefficient comparisons between compound sentiment and (1) closing price (2) change in price (closing price - opening price). Computed the Spearman rank correlation coefficient as well.

Compared effect of stemming and not stemming words on Pearson's and Spearman's correlation coefficient.


Conclusion:

Pearson:
Correlation between both (1) and (2) is negligible (<1%) without stemming. Stemming appears to improve correlation, but correlation is still very small (<2%)

Spearman:
Correlation for both (1) and (2) is still small, but better than Pearson. Stemming has inconsistent results, slightly lowering (2) but increasing (1)

Overall, Vader sentiment analysis produces very weak correlation with both (1) and (2). Try with other models.


Note: 

1. Try removing headlines from less common news sources (eg. headline count < 10). Unlikely to produce significant changes.

2. Might be helpful to determine the most relevant news sources by taking highly correlated news sources with instances of more than 20. Limiting the data might increase correlation. Use test set to evaluate if using this method.

3. Doing linear regression on neg, neu and pos score might produce interesting results.

In [52]:
df_final_unstem.loc[df_final_unstem['news']==' The Telegraph', ]

,ticker,date,time,headline,news,neg,neu,pos,compound,open,close,change
11629,COST,2020-12-30,01:14PM,"Tesco Britains worst minimum wage offender, BE...",The Telegraph,0.524,0.476,0.0,-0.765,373.462590,373.712097,0.249507
29538,MDLZ,2021-02-04,06:57AM,Cadbury brings Dairy Milk production back Bour...,The Telegraph,0.000,1.000,0.0,0.000,55.310001,56.000000,0.689999


Notes:

1. Take note of changes in the composition of S&P 500.